In [27]:
import cv2
import moviepy.editor as mp
import numpy as np

def visualize_frame_slicing(frame, num_horizontal_slices, num_vertical_slices, overlap_pixels):
    # Get frame dimensions
    height, width = frame.shape[:2]

    # Calculate patch dimensions
    patch_width = (width + overlap_pixels) // num_horizontal_slices
    patch_height = (height + overlap_pixels) // num_vertical_slices

    # Create an array to store the frames
    frames = []

    for i in range(num_horizontal_slices):
        for j in range(num_vertical_slices):
            # Calculate patch coordinates
            x1 = i * (patch_width - overlap_pixels)
            y1 = j * (patch_height - overlap_pixels)
            x2 = x1 + patch_width
            y2 = y1 + patch_height

            # Draw a red box around the patch
            frame_copy = frame.copy()
            cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 0, 255), 2)

            # Append the frame to the frames list
            frames.append(frame_copy)

    return frames

# Example usage
video_clip = mp.VideoFileClip("../input_dir/markt2_fight.mp4")

# Choose parameters for frame slicing
num_horizontal_slices = 4
num_vertical_slices = 3
overlap_pixels = 20

# Extract a specific frame (e.g., frame at time 5 seconds)
target_frame_time = 5
target_frame = video_clip.get_frame(target_frame_time)

# Visualize frame slicing for the target frame
frames_with_boxes = visualize_frame_slicing(target_frame, num_horizontal_slices, num_vertical_slices, overlap_pixels)

# Create a list of durations for each frame
durations = [1 / video_clip.fps] * len(frames_with_boxes)

# Create a video clip from the frames with correct durations
output_clip = mp.ImageSequenceClip(frames_with_boxes, fps=video_clip.fps, durations=durations)

# Save the video clip as a GIF
output_clip.write_gif("output.gif", fps=video_clip.fps)


MoviePy - Building file output.gif with imageio.


In [28]:
target_frame.shape

(1944, 2592, 3)

In [29]:
nr_patches = [2, 2]
overlaps = [300, 200]

model_size = 1280
model_half = True


conf_thres = 0.2
n_val_kpts = -1
kpt_thres = 0.65
iou_thres = 0.45
ios_thres = 0.5

In [30]:
import os

In [31]:
data = "markt2"

frame_number = 250 # indicate frame number of video if input is vdeo
# 250
data_samples = {"markt2": "../input_dir/markt2_fight.mp4", 
                "cam1": "../input_dir/cam1-3-a.mp4",
                "cam2": "../unput_dir/cam1-4.mp4"}


if os.path.splitext(data_samples[data])[-1][1:] in ('jpg', 'png'):
    frame = cv2.imread(data_samples[data]) # BGR format
elif os.path.splitext(data_samples[data])[-1][1:] in ('mp4'):
    cap = cv2.VideoCapture(data_samples[data])
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
    rval, frame = cap.read()
    cap.release()
    

stream_shape = frame.shape[:2]
print("stream shape:", stream_shape)

stream shape: (1944, 2592)


In [32]:
import torch
import torch.nn.functional as F
import numpy as np

def patches_yyxx(stream_shape,
                 nr_patches,
                 overlaps):
    """
    Compute y_1, y_2, x_1, x_2 coordinates of patches. 
    Args:
        stream_shape: list
            H and W of input stream.
        nr_patches: list of int
            nr of H and W patches.
        overlaops: list of int
            size of overlaps in H and W patches.
    Returns:
        patches_yyxx: list of int
            list of shape nr_patches[0]*nr_patches[1]: each element contains y1y2x1x2 of patches.
        ly,lx: list of int 
            stream_shape/nr_patches
        dy, dx: list of int
            half of overlps. The shape of patches will be ly+dy, lx+dx
            
        
    """
    if isinstance(nr_patches, int):
        nr_patches = [nr_patches, nr_patches]
    if isinstance(overlaps, int):
        overlaps = [overlaps, overlaps]
    
    dy = overlaps[0]/2
    dx = overlaps[1]/2

    ly = stream_shape[0]/nr_patches[0] 
    lx = stream_shape[1]/nr_patches[1]

    patches_yyxx = []
    for i in range(nr_patches[0]):
        for j in range(nr_patches[1]):
            y1 = int(max(i*ly - dy, 0))
            x1 = int(max(j*lx - dx, 0))
        
            y2 = int(min(y1 + ly + dy , stream_shape[0])) 
            x2 = int(min(x1 + lx + dx, stream_shape[1])) 
        
            patches_yyxx.append([y1,y2,x1,x2])
    
    return patches_yyxx, [ly,lx], [dy,dx]


In [33]:
patches_yyxx, lyx, dyx = patches_yyxx(stream_shape, nr_patches, overlaps)
ly, lx = lyx[0], lyx[1]
dy, dx = dyx[0], dyx[1]

crop_shape = patches_yyxx[0][1]-patches_yyxx[0][0], patches_yyxx[0][3]-patches_yyxx[0][2]
print("HxW of crops: {}x{}".format(crop_shape[0], crop_shape[1]))

HxW of crops: 1122x1396


In [34]:
from moviepy.editor import ImageSequenceClip
import os.path as osp


def get_output(frames, out_filename, fps=20, out_type='mp4'):
    video_clips = ImageSequenceClip(frames, fps=fps)
    out_type = osp.splitext(out_filename)[1][1:]
    if out_type == 'gif':
        video_clips.write_gif(out_filename)
    else:
        video_clips.write_videofile(out_filename, remove_temp=True)

In [38]:
sliding_patches = []
for i in range(0, nr_patches[0] * nr_patches[1]):
    frame_clone = frame.copy()
    cv2.rectangle(frame_clone,
                  (int(patches_yyxx[i][2]), int(patches_yyxx[i][0])),
                  (int(patches_yyxx[i][3]), int(patches_yyxx[i][1])),
                  color=(0, 0, 255),
                  thickness=8,
                  lineType=cv2.LINE_AA)
    text_sliding = str(nr_patches[0]) + '-' +  str(nr_patches[1]) + '_'+ str(overlaps[0]) + '-' + str(overlaps[1])
    cv2.putText(frame_clone, text_sliding,  
                (45, 90), cv2.FONT_HERSHEY_TRIPLEX, 2, 
                (0, 255, 0), 2)
    sliding_patches.append(cv2.cvtColor(frame_clone, cv2.COLOR_RGB2BGR))


file_sliding = text_sliding + '.mp4'
                  
path_sliding = '/work/output_dir/' + file_sliding
rel_path_sliding = '../output_dir/' + file_sliding


get_output(sliding_patches, path_sliding, fps=1, out_type='mp4')

Moviepy - Building video /work/output_dir/2-2_300-200.mp4.
Moviepy - Writing video /work/output_dir/2-2_300-200.mp4



Moviepy - Done !
Moviepy - video ready /work/output_dir/2-2_300-200.mp4
